In [57]:
Ruedas = {
    "base": {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "notch"    : "",
         "turnover" : ""
    },
    "I"  : {
         "secuencia": "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
         "notch"    : "Q",
         "turnover" : "R"
    },
    "II" : {
         "secuencia": "AJDKSIRUXBLHWTMCQGZNPYFVOE",
         "notch"    : "E",
         "turnover" : "F"
    },
    "III": {
         "secuencia": "BDFHJLCPRTXVZNYEIWGAKMUSQO",
         "notch"    : "V",
         "turnover" : "W"
    },
    "IV" : {
         "secuencia": "ESOVPZJAYQUIRHXLNFTGKDCMWB",
         "notch"    : "J",
         "turnover" : "K"
    },
    "V"  : {
         "secuencia": "VZBRGITYUPSDNHLXAWMJQOFECK",
         "notch"    : "Z",
         "turnover" : "A"
    },
    "VI" : {
         "secuencia": "JPGVOUMFYQBENHZRDKASXLICTW",
         "notch"    : "ZM",
         "turnover" : "AN"
    },
    "VII": {
         "secuencia": "NZJHGRCXMYSWBOUFAIVLPEKQDT",
         "notch"    : "ZM",
         "turnover" : "AN"
    },
    "VIII": {
         "secuencia": "FKQHTLXOCBJSPDZRAMEWNIUYGV",
         "notch"    : "ZM",
         "turnover" : "AN"
    }
}

Ruedas_griegas = {
    "base" : {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "notch"    : "",
         "turnover" : ""
     },
    "beta" : {
         "secuencia": "LEYJVCNIXWPBQMDRTAKZGFUHOS",
         "notch"    : "",
         "turnover" : ""
     },
    "gamma": {
         "secuencia": "FSOKANUERHMBTIYCWLQPZXVGJD",
         "notch"    : "",
         "turnover" : ""
     }
}

Reflectores = {
    "base"  : "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"     : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"     : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"     : "FVPJIAOYEDRZXWGCTKUQSBNMHL",
    "B_thin": "ENKQAUYWJICOPBLMDXZVFTHRGS",
    "C_thin": "RDOBJNTKVEHMLFCWZAXGYIPSUQ"
}

In [58]:
class Rotor:
    def __init__(self, rueda, ringstellung: str):      
        """
        Rueda es el mapa de la parte inicial del fichero. 
        """

        self.base     = Ruedas["base"]["secuencia"]
        self.secuencia   = rueda["secuencia"]
        self.notch    = rueda["notch"]
        self.turnover = rueda["turnover"]

        # Aplicamos el ringstellung, moviendo cada letra ciertas posiciones con respecto a la base
        indice_ring = Ruedas["base"]["secuencia"].find(ringstellung)
        self.secuencia = list(self.secuencia)
        
        for i in range(len(self.secuencia)):
            indice = Ruedas["base"]["secuencia"].find(self.secuencia[i]) + indice_ring
            self.secuencia[i] = Ruedas["base"]["secuencia"][indice % 26]
        
        cadena = ""
        for c in self.secuencia:
            cadena = cadena + c
        self.secuencia = cadena   
        self.secuencia = self.secuencia[-indice_ring:] + self.secuencia[:-indice_ring]


    def siguiente(self, indice: int):
        letra_a_buscar   = self.secuencia[indice]
        indice_siguiente = self.base.index(letra_a_buscar)
        return indice_siguiente

    def revertir(self, indice):
        letra_en_base    = self.base[indice]
        indice_de_salida = self.secuencia.index(letra_en_base)
        return indice_de_salida
    
    def rotar(self):
        self.base   = self.base[1:] + self.base[:1]
        self.secuencia = self.secuencia[1:] + self.secuencia[:1]
    
    def ajusta_grundstellung(self, indice):
        self.base   = self.base[indice:] + self.base[:indice]
        self.secuencia = self.secuencia[indice:] + self.secuencia[:indice]

    def ha_tocado_notch(self):
        return self.base[0] in self.notch

In [59]:
class Reflector:
    def __init__(self, reflector: str):
        self.secuencia = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.secuencia[indice]
        indice_siguiente = Reflectores["base"].find(letra_a_buscar)
        return indice_siguiente

In [60]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[letra] = letra

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    def siguiente(self, letra: str):
        return self.mapping[letra]

In [94]:
class Enigma:
    def __init__(self, ruedas, ringstellungs: str, grundstellungs: str, reflector: str, steckern: list[(str, str)]):
        
        if len(ruedas) != len(ringstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del ringstellung")
        if len(ruedas) != len(grundstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del grundstellung")
        
        self.rotores = []
        for rotor, ring in zip(ruedas, ringstellungs):
            self.rotores.append(Rotor(rotor, ring))
        
        # Encontramos los índices correspondientes al grundstellung
        indices = []
        for letra in grundstellungs:
            indices.append(Ruedas["base"]["secuencia"].find(letra))

        for rotor, indice in zip(self.rotores, indices):
            rotor.ajusta_grundstellung(indice)

        self.plugboard = Plugboard(steckern)
        self.reflector = Reflector(reflector)

        self.needs_rotation = [True, False, False, False]
    
    def cifrar_caracter(self, c: str):
        # Letra -> plugbooard -> rotores -> reflector -> rotores -> plugboard -> salida
        letra = self.plugboard.siguiente(c)

        # Vigilar cuándo toca el notch, para girarlos en la siguiente vez que se introduzca una letra.     
        self.needs_rotation[1] = self.rotores[0].ha_tocado_notch()
        self.needs_rotation[2] = self.rotores[1].ha_tocado_notch()

        for rotor, do_i_rotate in zip(self.rotores, self.needs_rotation):
            if do_i_rotate:
                rotor.rotar()
        
        # Caso especial: double step
        if self.needs_rotation[2]:
            self.rotores[1].rotar()

        # A partir de aquí, necesitamos los índices, que nos serán más cómodos. Retomaremos la letra más tarde
        indice_letra = Ruedas["base"]["secuencia"].find(letra)

        for rotor in self.rotores:
            indice_letra = rotor.siguiente(indice_letra)
        
        indice_letra = self.reflector.reflejar(indice_letra)
    
        for rotor in reversed(self.rotores):
            indice_letra = rotor.revertir(indice_letra)
        
        letra = Ruedas["base"]["secuencia"][indice_letra]

        letra = self.plugboard.siguiente(letra)
        
        return letra

    def cifrar(self, mensaje: str):
        # FIXME cuidado con la configuración inicial
        
        salida = ""

        for letra in mensaje:
            if letra == " ":
                salida = salida + " "
            else:
                salida = salida + self.cifrar_caracter(letra)
        
        return salida


In [100]:
rotores = [Ruedas["VIII"], Ruedas["VI"], Ruedas["V"], Ruedas_griegas["beta"]]
ringstellungs = "LEPE"   # 11 4 15 4
grundstellungs_iniciales = "MEAN"  # 12 4 0 13
clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]

enigma = Enigma(rotores, ringstellungs, grundstellungs_iniciales, Reflectores["C_thin"], clavijas)

futuro_grundstellungs = "QEOB"
grundstellungs = enigma.cifrar(futuro_grundstellungs)
print(futuro_grundstellungs)
print(grundstellungs)

# Configuramos la nueva Enigma con el grundstellung que hemos obtenido
enigma = Enigma(rotores, ringstellungs, grundstellungs[::-1], Reflectores["C_thin"], clavijas)

mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"
# Quitar 2 primeros y 2 últimos grupos, que sirven para advertir fallos en la transmisión        
mensaje = mensaje[10:len(mensaje)-10]

mensaje_cifrado = enigma.cifrar(mensaje)

print(f"Mensaje original: {mensaje}")
print(f"Mensaje cifrado: {mensaje_cifrado}")

mensaje_museo = "KRKRALLEXXFOLGENDESISTSOFORTBEKANNTZUGEBENXXICHHABEFOLGELNBEBEFEHLERHALTENXXJANSTERLEDESBISHERIGXNREICHSMARSCHALLSJGOERINGJSETZTDERFUEHRERSIEYHVRRGRZSSADMIRALYALSSEINENNACHFOLGEREINXSCHRIFTLSCHEVOLLMACHTUNTERWEGSXABSOFORTSOLLENSIESAEMTLICHEMASSNAHMENVERFUEGENYDIESICHAUSDERGEGENWAERTIGENLAGEERGEBENXGEZXREICHSLEITEIKKTULPEKKJBORMANNJXXOBXDXMMMDURNHFKSTXKOMXADMXUUUBOOIEXKP"
mensaje_cifrado.replace(" ", "")
print(mensaje_cifrado.replace(" ", "") == mensaje_museo)

QEOB
CDSZ
Mensaje original: LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW
Mensaje cifrado: KRKR ALLE XXFO LGEN DESI STSO FORT BEKA NNTZ UGEB ENXX ICHH ABEF OLGE LNBE BEFE HLER HALT ENXX JANS TERL EDES BISH ERIG XNRE ICHS MARS CHAL LSJG OERI NGJS ETZT DERF UEHR ERSI EYHV RRGR ZSSA DMIR ALYA LSSE INEN NACH FOLG EREI NXSC HRIF TLSC HEVO LLMA CHTU NTER WEGS XABS OFOR TSOL LENS IESA EMTL ICHE MASS NAHM ENVE RFUE GENY DIES ICHA USDE RGEG ENWA ERTI GENL AGEE RGEB ENXG EZXR EICH SLEI TEIK KTUL PEKK JBOR MANN JXXO BXDX MMMD URNH FKST XKOM XADM XUUU BOOI EXKP
True


In [63]:
print(Ruedas["V"]["secuencia"])

VZBRGITYUPSDNHLXAWMJQOFECK
